# **Cài đặt môi trường**

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.0 MB/s eta 0:00:00


In [2]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.0 MB/s eta 0:00:00


In [3]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
# Load model bloomz-1b1
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b1").to("cuda")
# Load tokenizer từ bloomz-1b1
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")

# **Tải dataset hỏi đáp web bán hàng**

In [5]:
!wget https://raw.githubusercontent.com/phatjkk/data/main/LLM/Ecommerce_FAQ_Chatbot_dataset.json

--2023-07-12 15:43:59--  https://raw.githubusercontent.com/phatjkk/data/main/LLM/Ecommerce_FAQ_Chatbot_dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20978 (20K) [text/plain]
Saving to: ‘Ecommerce_FAQ_Chatbot_dataset.json’

Ecommerce_FAQ_Chatb 100%[===================>]  20.49K  --.-KB/s    in 0s      

2023-07-12 15:44:00 (113 MB/s) - ‘Ecommerce_FAQ_Chatbot_dataset.json’ saved [20978/20978]



In [6]:
from datasets import load_dataset
from datasets import Dataset
data = load_dataset('json', data_files='Ecommerce_FAQ_Chatbot_dataset.json')
ds = Dataset.from_list(data["train"]["questions"][0])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-db621cb92fe7129c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Kết hợp câu hỏi và câu trả lời liền kề nhau để model học nội dung
# Mục đích là để khi đưa 1 đoạn câu hỏi, model sẽ tự nhiểu và gen ra câu trả lời
# phía sau
def merge_columns(example):
    example["prediction"] = example["question"] + " ->: " + str(example["answer"])
    return example

In [8]:
# Map function trên vào dataset
ds = ds.map(merge_columns)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [9]:
# Tokenizer/Véc tơ hóa văn bản (Chuyển văn bản thành số để training)
def tokeni(example):
    example["prediction_token"] = tokenizer(example["prediction"], return_tensors='pt', padding=True)['input_ids']
    return example
# Map function trên vào dataset
ds = ds.map(tokeni,batched=True)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [10]:
# Hàm tính số lượng parameters có thể train được trên model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [11]:
# Tạo LoraConfig để áp dụng Lora vào việc training mô hình
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, #attention heads
    lora_alpha=16, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model_lora = get_peft_model(model, config)
print_trainable_parameters(model_lora)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

trainable params: 1179648 || all params: 1066493952 || trainable%: 0.11060990995662018


In [12]:
# Training model bằng hàm trainer
import transformers
trainer = transformers.Trainer(
    model=model_lora,
    train_dataset=ds["prediction_token"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    )
    ,data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model_lora.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,3.713500
2,3.201300
3,3.318000
4,3.348100
5,2.610100
6,2.982500
7,3.450500
8,3.185200
9,3.299900
10,3.966300


TrainOutput(global_step=100, training_loss=2.3110049557685852, metrics={'train_runtime': 13.9389, 'train_samples_per_second': 14.348, 'train_steps_per_second': 7.174, 'total_flos': 49361018167296.0, 'train_loss': 2.3110049557685852, 'epoch': 2.5})

In [13]:
tokenizer

BloomTokenizerFast(name_or_path='bigscience/bloomz-1b1', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [14]:
print(ds["question"][0])
print(ds["answer"][0])

How can I create an account?
To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.


In [15]:
question = str(ds["question"][0]+" ->: ")
question

'How can I create an account? ->: '

In [16]:
# In ra kết quả
inputs = tokenizer( question, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=1000)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

How can I create an account? ->:  If you do not have an account, please create one. Once you have created an account, you will be able to access your account and perform other activities.


In [17]:
question2 = str(ds["question"][1]+" ->: ")
question2

'What payment methods do you accept? ->: '

In [18]:
# In ra kết quả
inputs = tokenizer( question2, return_tensors="pt")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

What payment methods do you accept? ->:  You can accept payment methods such as credit cards, debit cards, and e-wallets. Please note that payment methods are subject to restrictions and may not be available for certain transactions. Please review the payment method details for more information.


In [19]:
ask = "How to login?"
question3 = str(ask+" ->: ")
inputs = tokenizer( question3, return_tensors="pt")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

How to login? ->:  You can log in to your account by entering your username and password. If you have forgotten your password, you can reset it by clicking the "Reset Password" link. If you are logged into your account, you can also log out by clicking the "Log Out" link.
